## Dropoutありのモデルの定義
nn.Dropoutを利用

In [1]:
import random

import torch
import torch.utils.data
from torch import nn
from torch import optim

class MultiLayer(nn.Module):
    """
    多層モデルクラス
    """
    def __init__(self):
        super(MultiLayer, self).__init__()
        self.linear1 = torch.nn.Linear(10, 1) # wx+b 初期パラメータはランダム

        # dropoutの追加
        self.dropout = nn.Dropout(p=0.5) # p:dropout対象のパラメータの割合
        
        # 損失関数
        self.loss_func = nn.MSELoss()

        
    def forward(self, input_vector):
        """
        順伝搬
        :param input_vector: 入力ベクトル
        :return tanh_w3x: モデルの出力
        """
        w1x = self.linear1(input_vector)
        print("dropout前: ",w1x)
        w1x = self.dropout(w1x)
        print("dropout後: ",w1x)
        return w1x

    
    def forward_loss(self, input_vector, target_values):
        """
        順伝搬 + 損失計算
        :param input_vector: 入力ベクトル
        :param target_values: 正解の値
        :return loss:損失
        """
        wx = self.forward(input_vector)
        loss = self.loss_func(wx, target_values)
        return loss


In [2]:
# モデルのインスタンス
multi_layer_model = MultiLayer()

### 学習

In [5]:
# 損失の最小化のための最適化手法
op = optim.SGD(multi_layer_model.parameters(), lr=0.1) # lr:learning rate (学習率)

# 入力とラベル
input_vector = torch.rand(50, 10)
target_values = torch.reshape(torch.tensor([[i[0]+1 for i in input_vector]]), (50, 1))
train_data = torch.utils.data.TensorDataset(input_vector, target_values)
train_data_size = len(train_data)

# ミニバッチサイズ
minibatch_size = 10
max_batch_no = train_data_size // minibatch_size

# 学習
# エポック
for epoch in range(3):
    print("epoch: ", epoch)

    # ミニバッチ学習のためのデータローダー
    train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=minibatch_size, shuffle=True)

    # ミニバッチごとに逆伝搬とパラメータ更新
    # イテレーション
    for batch_no, (batch_input_vector, batch_target_values) in enumerate(train_data_loader):            
        # 順伝搬と損失計算
        loss = multi_layer_model.forward_loss(batch_input_vector, batch_target_values)
        print("batch_no: {}/{} loss: {}".format(batch_no+1, max_batch_no, loss))


        # 逆伝搬 (勾配の設定)
        loss.backward()

        # パラメータの更新
        op.step()

        # 勾配の消去
        multi_layer_model.zero_grad()


    print("="*10)

epoch:  0
dropout前:  tensor([[0.5999],
        [0.8199],
        [0.8084],
        [0.6593],
        [0.7123],
        [0.6373],
        [0.5845],
        [0.4164],
        [0.6212],
        [0.6483]], grad_fn=<AddmmBackward>)
dropout後:  tensor([[0.0000],
        [0.0000],
        [1.6168],
        [0.0000],
        [1.4247],
        [0.0000],
        [1.1690],
        [0.8327],
        [1.2423],
        [1.2967]], grad_fn=<MulBackward0>)
batch_no: 1/5 loss: 1.0170060396194458
dropout前:  tensor([[0.5016],
        [0.8467],
        [0.7798],
        [1.0743],
        [0.9552],
        [0.6635],
        [0.9582],
        [0.6690],
        [1.0536],
        [1.0290]], grad_fn=<AddmmBackward>)
dropout後:  tensor([[1.0032],
        [1.6934],
        [1.5595],
        [0.0000],
        [0.0000],
        [1.3270],
        [0.0000],
        [1.3380],
        [2.1072],
        [0.0000]], grad_fn=<MulBackward0>)
batch_no: 2/5 loss: 1.2145252227783203
dropout前:  tensor([[1.0138],
        [0.7964],